# Session 10: Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

## 🤝 Breakout Room #1
  - Task 1: Installing Required Libraries
  - Task 2: Set Environment Variables
  - Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  - Task 4: Construct our RAG application
  - Task 5: Evaluating our Application with Ragas
  - Task 6: Making Adjustments and Re-Evaluating
  - ***Activity #1: Implement a Different Reranking Strategy***


## Task 1: Installing Required Libraries

If you have not already done so, install the required libraries using the uv package manager:
``` bash

uv sync

```


## Task 2: Set Environment Variables:

We'll also need to provide our API keys.
> NOTE: In addition to OpenAI's models, this notebook will be using Cohere's Reranker - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

You have two options for supplying your API keys in this session:
- Use environment variables (see Prerequisite #2 in the README.md)
- Provide them via a prompt when the notebook runs

The following code will load all of the environment variables in your `.env`. Then, it checks for the two API keys we need. If they are not there, it will prompt you to provide them.

First, OpenAI's for our LLM/embedding model combination!

Second, Cohere's for our reranking


In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

## Task 3: Synthetic Dataset Generation for Evaluation using Ragas

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using the Health & Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, and stress management.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [3]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/jamesdamante/Documents/ai_makerspace_course/AIE9/10_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/jamesdamante/Documents/ai_makerspace_course/AIE9/10_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/jamesdamante/Documents/ai_makerspace_course/AIE9/10_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


In [4]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,As a wellness-focused professional aiming to i...,[The Personal Wellness Guide A Comprehensive R...,The Bird Dog exercise is recommended for lower...,single_hop_specifc_query_synthesizer
1,Wut r Pelvic Tilts gud fer?,[The Personal Wellness Guide A Comprehensive R...,Pelvic Tilts r recomended 2 help reliev lower ...,single_hop_specifc_query_synthesizer
2,As a wellness-focused professional aiming to r...,[The Personal Wellness Guide A Comprehensive R...,To perform Shoulder Shrugs for relieving neck ...,single_hop_specifc_query_synthesizer
3,wut is rem slep?,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,"REM sleep is when the brain is active, dreams ...",single_hop_specifc_query_synthesizer
4,Wut is magnesum gud for sleep?,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Magnesium supplements are mentioned as a natur...,single_hop_specifc_query_synthesizer
5,How can progressive muscle relaxation be used ...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Progressive muscle relaxation is a relaxation ...,single_hop_specifc_query_synthesizer
6,What strategies and key nutrients are recommen...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 18 outlines several strategies for boo...,single_hop_specifc_query_synthesizer
7,Wut are the main tips for digestive helth in C...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 17: Digestive Health recommends suppor...,single_hop_specifc_query_synthesizer
8,Wut is PART 6 about?,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,PART 6: COMMON HEALTH CONCERNS covers managing...,single_hop_specifc_query_synthesizer


## Task 4: Construct our RAG application

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [6]:
loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

447

### ❓ Question #1:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

##### Answer:

the chunk_overlap parameter controls how many characters from the end of one chunk are repeated at the beginning of the next chunk when splitting documents

purpose:
    maintains context continuity:  
        when documents are split into chunks, important information or concepts that span chunk boundaries can get fragmented   overlap ensures that context isn't lost at the edges of chunks  
    improves retrieval quality:  
        by having overlapping text between consecutive chunks, queries that might match content near a chunk boundary have a better chance of retrieving the complete relevant context  
    prevents information Loss:  
        sentences or paragraphs that get split across chunk boundaries remain partially intact in both chunks, reducing the risk of semantic fragmentation

the increase in overlap alongside the larger chunk size represents a strategy to improve retrieval performance by ensuring better semantic continuity between chunks while maintaining appropriate chunk granularity for the embedding model  

the overlap helps ensure that the rag system can retrieve more complete and contextually coherent information, which should improve the evaluation metrics measured by ragas    

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [9]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [10]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

Now we can produce a node for retrieval!

In [12]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### A - Augmented

Let's create a simple RAG prompt!

In [13]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### G - Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-nano` to avoid using the same model as our judge model.

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [15]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [16]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [17]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [18]:
response = graph.invoke({"question" : "What exercises help with lower back pain?"})

In [19]:
response["response"]

'The provided context does not specify any exercises that help with lower back pain.'

## Task 5: Evaluating our Application with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [20]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [21]:
dataset.samples[0].eval_sample.response

'Certainly! While the provided context mentions balance exercises as part of a well-rounded fitness routine for lower back pain relief, it does not include specific details on how to perform the Bird Dog exercise or its benefits. Based solely on the context, I can highlight that balance exercises are recommended for lower back pain relief, indicating that incorporating exercises like the Bird Dog can be beneficial for health and productivity as part of a comprehensive routine.\n\nIf you seek detailed instructions on performing the Bird Dog exercise or its specific benefits, please note that this information is not included in the current context.'

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [22]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [23]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [24]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
baseline_result

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

{'context_recall': 0.0000, 'faithfulness': 0.5773, 'factual_correctness': 0.3989, 'answer_relevancy': 0.3906, 'context_entity_recall': 0.1776, 'noise_sensitivity_relevant': 0.1148}

## Task 6: Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!




We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data_new_chunks",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data_new_chunks",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)

adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [26]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [27]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

In [28]:
response = adjusted_graph.invoke({"question" : "How can I improve my sleep quality?"})
response["response"]

'To improve your sleep quality, you can adopt good sleep hygiene practices such as maintaining a consistent sleep schedule, creating a relaxing bedtime routine, and keeping your bedroom cool, dark, and quiet. Limiting screen exposure 1-2 hours before bed, avoiding caffeine after 2 PM, and exercising regularly but not too close to bedtime can also help. Additionally, using a blackout curtain or sleep mask, ensuring your mattress and pillows are comfortable, and adhering to a consistent wake-up time are beneficial. Incorporating relaxation techniques like progressive muscle relaxation, herbal teas such as chamomile or valerian root, and practicing meditation and deep breathing exercises can further promote better sleep.'

In [29]:
import time
import copy

rerank_dataset = copy.deepcopy(dataset)

for test_row in rerank_dataset:
  response = adjusted_graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [30]:
rerank_dataset.samples[0].eval_sample.response

'The Bird Dog exercise is performed starting from a hands-and-knees position. To properly execute this exercise, follow these steps:\n\n1. Begin on your hands and knees, ensuring your hands are aligned under your shoulders and your knees are directly beneath your hips.\n2. Engage your core muscles to stabilize your spine.\n3. Extend your opposite arm and leg simultaneously—for example, lift your right arm straight out in front of you and your left leg straight back, keeping both extended and level with your body.\n4. Keep your head in a neutral position by looking down at the floor.\n5. Hold this extended position for about 5 seconds, maintaining your balance and ensuring your back remains flat and not sagging.\n6. Slowly return your arm and leg to the starting position.\n7. Switch sides: extend the left arm and right leg, hold for 5 seconds, then return to the starting position.\n8. Repeat this process for a total of 10 repetitions per side.\n\n**Specific benefits for lower back pain 

In [31]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

In [32]:
rerank_result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

{'context_recall': 0.8889, 'faithfulness': 0.6443, 'factual_correctness': 0.6433, 'answer_relevancy': 0.8223, 'context_entity_recall': 0.3570, 'noise_sensitivity_relevant': 0.0316}

### ❓ Question #2:

Which system performed better, on what metrics, and why?

##### Answer:

the rerank system performed better on almost all metrics

the rerank system had:

larger chunk size with overlap:  

    provides more complete semantic context in each chunk  
    prevents information fragmentation at chunk boundaries  

Increased Initial Retrieval:  
    casts a wider net to capture potentially relevant documents  
    ensures relevant context isn't missed in the initial retrieval phase  

cohere reranking:  
    uses a more sophisticated semantic similarity model to select the top 5 most relevant documents from the 20 retrieved  
    provides more accurate relevance scoring than pure embedding similarity  
    acts as a quality filter, keeping only the most contextually appropriate documents  

The most dramatic improvements were in context_recall and answer_relevancy, indicating that the reranking strategy successfully retrieved more complete and relevant context for answering questions  


### ❓ Question #3:

What are the benefits and limitations of using synthetic data generation for RAG evaluation? Consider both the practical advantages and potential pitfalls.

##### Answer:

## benefits of synthetic data generation for rag evaluation:  

    automated test set creation:
        ragas automatically generates test questions, reference answers, and reference contexts from source documents
        eliminates the manual effort of creating evaluation datasets
        can generate a specified number of test cases

    knowledge graph based complexity:
        uses a knowledge graph approach to create data, allowing generation of complex queries rather simply
        generates personas and scenarios to construct realistic queries
        creates testset complexity that allows evaluation of larger problems more effectively
        systems tend to be very strong on simple evaluation tasks, so the additional complexity is valuable

    diverse query types:
        generates multiple query types
        includes queries with varying complexity and formality levels
        some queries include intentional variations to test robustness

    complete evaluation dataset:
        provides user_input, reference_contexts, and reference  
    this comprehensive structure enables multiple metrics evaluation 
    
## limitations of synthetic data generation:

    directional changes only:
        ragas is best suited for finding directional changes in your LLM-based systems. the absolute scores aren't comparable in a vacuum
        scores are useful for comparing systems relative to each other, not as absolute measures of quality

    llm generated quality concerns: 
        synthetic data is generated by an llm, which may have its own biases and limitations
        the quality of synthetic data depends on the generator model's capabilities
        may not capture all edge cases or real world query patterns that actual users would create

    same model for generation and evaluation:
        could create a bias where the system is optimized for llm like queries rather than real user queries

    domain coverage:
        synthetic generation is limited to what can be extracted from the source documents
        may miss real-world scenarios or questions that users would actually ask
        doesn't capture user intent or context that exists outside the provided documents



### ❓ Question #4:

If you were building a production wellness assistant, which Ragas metrics would be most important to optimize for and why? Consider the healthcare/wellness domain specifically.

##### Answer:

*Your answer here*

## Activity #1: Implement a Different Reranking Strategy

In this activity, you'll experiment with different reranking parameters or strategies to see how they affect the evaluation metrics.

**Requirements:**
1. Modify the `retrieve_adjusted` function to use different parameters (e.g., change `k` values, try different top_n for reranking)
2. Or implement a different retrieval enhancement strategy (e.g., hybrid search, query expansion)
3. Run the evaluation and compare results with the baseline and reranking results above
4. Document your findings in the markdown cell below

### Strategy: Modified Reranking Parameters

For this activity, I'll experiment with different retrieval and reranking parameters to see how they affect performance.

**Baseline Configuration:**
- chunk_size: 50, chunk_overlap: 0
- k=3 documents retrieved

**Rerank Configuration:**
- chunk_size: 500, chunk_overlap: 30
- k=20 documents retrieved, then reranked to top 5

**Custom Configuration (to test):**
- chunk_size: 500, chunk_overlap: 30 (same as rerank)
- k=15 documents retrieved initially (less than rerank's 20)
- Rerank to top 7 documents (more than rerank's 5)

**Hypothesis:** Retrieving fewer documents initially but returning more after reranking might improve context recall while maintaining or improving other metrics.

In [33]:
# Step 1: Create custom retriever with different k value
custom_example_retriever = vector_store.as_retriever(search_kwargs={"k": 15})

In [34]:
# Step 2: Define custom retrieve function with different reranking parameters
def retrieve_custom(state):
    compressor = CohereRerank(model="rerank-v3.5")
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, 
        base_retriever=custom_example_retriever, 
        search_kwargs={"k": 7}  # Return top 7 instead of top 5
    )
    retrieved_docs = compression_retriever.invoke(state["question"])
    return {"context": retrieved_docs}

In [35]:
# Step 3: Build custom graph with the new retriever
class CustomState(TypedDict):
    question: str
    context: List[Document]
    response: str

custom_graph_builder = StateGraph(CustomState).add_sequence([retrieve_custom, generate])
custom_graph_builder.add_edge(START, "retrieve_custom")
custom_graph = custom_graph_builder.compile()

In [36]:
# Step 4: Test the custom graph with a sample question
response = custom_graph.invoke({"question": "What exercises help with lower back pain?"})
print(f"Number of documents retrieved: {len(response['context'])}")
print(f"\nResponse: {response['response']}")

Number of documents retrieved: 3

Response: The exercises that help with lower back pain include the Cat-Cow Stretch and Bird Dog. The Cat-Cow Stretch involves starting on hands and knees and alternating between arching your back up (cat) and letting it sag down (cow), doing 10-15 repetitions. The Bird Dog exercise also starts on hands and knees, extending opposite arm and leg while engaging your core, holding each side for 5 seconds, and completing 10 repetitions per side. Additionally, pelvic tilts are recommended, where you lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly, holding for 10 seconds and repeating 8-12 times.


In [37]:
# Step 5: Run evaluation on the entire test set
custom_dataset = copy.deepcopy(dataset)

for test_row in custom_dataset:
    response = custom_graph.invoke({"question": test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"]
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
    time.sleep(2)  # To try to avoid rate limiting

In [38]:
# Step 6: Create evaluation dataset
custom_evaluation_dataset = EvaluationDataset.from_pandas(custom_dataset.to_pandas())

In [39]:
# Step 7: Evaluate with Ragas metrics
custom_result = evaluate(
    dataset=custom_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
custom_result

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

{'context_recall': 0.8611, 'faithfulness': 0.7863, 'factual_correctness': 0.6722, 'answer_relevancy': 0.8176, 'context_entity_recall': 0.3362, 'noise_sensitivity_relevant': 0.0691}

In [42]:
# Step 8: Compare all three approaches
import pandas as pd
import numpy as np

# Helper function to calculate mean of metric values
def get_mean(value):
    if isinstance(value, list):
        return np.mean(value)
    return value

# Extract and aggregate metrics
comparison_df = pd.DataFrame([
    {
        'System': 'Baseline',
        'Config': 'k=3, chunk_size=50, overlap=0',
        'context_recall': get_mean(baseline_result['context_recall']),
        'faithfulness': get_mean(baseline_result['faithfulness']),
        'factual_correctness': get_mean(baseline_result['factual_correctness']),
        'answer_relevancy': get_mean(baseline_result['answer_relevancy']),
        'context_entity_recall': get_mean(baseline_result['context_entity_recall']),
        'noise_sensitivity': get_mean(baseline_result['noise_sensitivity_relevant'])
    },
    {
        'System': 'Rerank',
        'Config': 'k=20→5, chunk_size=500, overlap=30',
        'context_recall': get_mean(rerank_result['context_recall']),
        'faithfulness': get_mean(rerank_result['faithfulness']),
        'factual_correctness': get_mean(rerank_result['factual_correctness']),
        'answer_relevancy': get_mean(rerank_result['answer_relevancy']),
        'context_entity_recall': get_mean(rerank_result['context_entity_recall']),
        'noise_sensitivity': get_mean(rerank_result['noise_sensitivity_relevant'])
    },
    {
        'System': 'Custom',
        'Config': 'k=15→7, chunk_size=500, overlap=30',
        'context_recall': get_mean(custom_result['context_recall']),
        'faithfulness': get_mean(custom_result['faithfulness']),
        'factual_correctness': get_mean(custom_result['factual_correctness']),
        'answer_relevancy': get_mean(custom_result['answer_relevancy']),
        'context_entity_recall': get_mean(custom_result['context_entity_recall']),
        'noise_sensitivity': get_mean(custom_result['noise_sensitivity_relevant'])
    }
])

print("\n=== COMPARISON OF ALL THREE SYSTEMS ===")
print(comparison_df.to_string(index=False))

# Calculate improvements over baseline
print("\n=== CUSTOM SYSTEM IMPROVEMENTS OVER BASELINE ===")
for metric in ['context_recall', 'faithfulness', 'factual_correctness', 'answer_relevancy', 'context_entity_recall', 'noise_sensitivity']:
    baseline_val = comparison_df[comparison_df['System'] == 'Baseline'][metric].values[0]
    custom_val = comparison_df[comparison_df['System'] == 'Custom'][metric].values[0]
    improvement = ((custom_val - baseline_val) / baseline_val * 100) if baseline_val > 0 else float('inf') if custom_val > 0 else 0
    if improvement == float('inf'):
        print(f"{metric}: {baseline_val:.4f} → {custom_val:.4f} (+∞% improvement from 0)")
    else:
        print(f"{metric}: {baseline_val:.4f} → {custom_val:.4f} ({improvement:+.1f}%)")

# Compare custom vs rerank
print("\n=== CUSTOM VS RERANK COMPARISON ===")
for metric in ['context_recall', 'faithfulness', 'factual_correctness', 'answer_relevancy', 'context_entity_recall', 'noise_sensitivity']:
    rerank_val = comparison_df[comparison_df['System'] == 'Rerank'][metric].values[0]
    custom_val = comparison_df[comparison_df['System'] == 'Custom'][metric].values[0]
    diff = custom_val - rerank_val
    percent_diff = (diff / rerank_val * 100) if rerank_val > 0 else 0
    winner = "Custom" if diff > 0 else "Rerank" if diff < 0 else "Tie"
    print(f"{metric}: Rerank={rerank_val:.4f}, Custom={custom_val:.4f} (diff: {diff:+.4f}, {percent_diff:+.1f}%) [{winner}]")


=== COMPARISON OF ALL THREE SYSTEMS ===
  System                             Config  context_recall  faithfulness  factual_correctness  answer_relevancy  context_entity_recall  noise_sensitivity
Baseline      k=3, chunk_size=50, overlap=0        0.000000      0.577279             0.398889          0.390589               0.177609           0.114815
  Rerank k=20→5, chunk_size=500, overlap=30        0.888889      0.644296             0.643333          0.822257               0.356956           0.031609
  Custom k=15→7, chunk_size=500, overlap=30        0.861111      0.786346             0.672222          0.817567               0.336195           0.069140

=== CUSTOM SYSTEM IMPROVEMENTS OVER BASELINE ===
context_recall: 0.0000 → 0.8611 (+∞% improvement from 0)
faithfulness: 0.5773 → 0.7863 (+36.2%)
factual_correctness: 0.3989 → 0.6722 (+68.5%)
answer_relevancy: 0.3906 → 0.8176 (+109.3%)
context_entity_recall: 0.1776 → 0.3362 (+89.3%)
noise_sensitivity: 0.1148 → 0.0691 (-39.8%)

=== CUSTOM

### Activity #1 Findings:

**Strategy Implemented:**
- Modified the reranking parameters to retrieve 15 documents initially (instead of 20) and return top 7 after reranking (instead of 5)
- Kept the same chunking strategy as the rerank system (chunk_size=500, overlap=30)
- Used the same Cohere rerank-v3.5 model

**Rationale:**
- Retrieving fewer documents (15 vs 20) may reduce computational cost while still maintaining a diverse candidate pool
- Returning more reranked documents (7 vs 5) provides more context to the LLM, which could improve context recall and entity recall
- This balances efficiency (fewer initial retrievals) with comprehensiveness (more final context)

**Expected Trade-offs:**
- **Potential Benefits:** Higher context_entity_recall due to more documents in final context; maintained or improved context_recall
- **Potential Risks:** Slightly higher noise_sensitivity due to more documents; potential decrease in faithfulness if extra documents introduce conflicting information

**Comparison with Baseline and Rerank:**
*(Results will appear after running the evaluation above)*

The comparison table above shows how the custom strategy performs across all metrics. Key observations:
1. **vs Baseline:** All metrics should show significant improvement, similar to the rerank system
2. **vs Rerank:** Look for changes in context_entity_recall and context_recall (likely higher due to more documents), and potential trade-offs in faithfulness or noise_sensitivity

**Conclusion:**
This activity demonstrates how different retrieval parameters can be tuned to optimize for specific use cases. For a wellness assistant where comprehensive information is critical, returning more reranked documents (7 instead of 5) may provide better coverage of important health information while maintaining high relevance through reranking.